In [1]:
#!pip uninstall -y curl_cffi
!pip uninstall yfinance
#!pip install yfinance

^C


In [2]:

!pip install Pandas
!pip install requests
!pip install bs4
!pip install matplotlib 
!pip install seaborn

In [3]:
!pip install yfinance

In [4]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib as mlp
import os
os.environ["YFINANCE_USE_CURL_CFFI"] = "0"


We want to extract stock price from Yahoo Finance, so we need to make a ticker obj.
Each public company has a unique ticker symbol (e.g., Apple = AAPL)

In [5]:
apple = yf.Ticker("AAPL")
microsoft = yf.Ticker("MSFT")
google = yf.Ticker("GOOGL")

# Checking the information of stock price
microsoft_info = microsoft.info
microsoft_info["country"]
#microsoft_info["sector"]

'United States'

Using the ticker object and the function "history", we can extract stock information and save it in a dataframe

In [6]:
apple_data = apple.history(period = "max")
apple_data.reset_index(inplace=True)

mic_data = microsoft.history(period = "max")
mic_data.reset_index(inplace =True)

google_data = google.history(period ="max")
google_data.reset_index(inplace =True)


In [7]:
apple_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1980-12-12 00:00:00-05:00,0.098597,0.099025,0.098597,0.098597,469033600,0.0,0.0
1,1980-12-15 00:00:00-05:00,0.093881,0.093881,0.093453,0.093453,175884800,0.0,0.0
2,1980-12-16 00:00:00-05:00,0.087022,0.087022,0.086594,0.086594,105728000,0.0,0.0
3,1980-12-17 00:00:00-05:00,0.088737,0.089165,0.088737,0.088737,86441600,0.0,0.0
4,1980-12-18 00:00:00-05:00,0.091310,0.091738,0.091310,0.091310,73449600,0.0,0.0


In [8]:
mic_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1986-03-13 00:00:00-05:00,0.054277,0.062259,0.054277,0.059598,1031788800,0.0,0.0
1,1986-03-14 00:00:00-05:00,0.059598,0.062791,0.059598,0.061726,308160000,0.0,0.0
2,1986-03-17 00:00:00-05:00,0.061726,0.063323,0.061726,0.062791,133171200,0.0,0.0
3,1986-03-18 00:00:00-05:00,0.062791,0.063323,0.060662,0.061194,67766400,0.0,0.0
4,1986-03-19 00:00:00-05:00,0.061194,0.061726,0.059598,0.060130,47894400,0.0,0.0


In [9]:
google_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2004-08-19 00:00:00-04:00,2.487584,2.588579,2.387085,2.496041,893181924,0.0,0.0
1,2004-08-20 00:00:00-04:00,2.512708,2.713456,2.500021,2.694301,456686856,0.0,0.0
2,2004-08-23 00:00:00-04:00,2.754999,2.822910,2.712710,2.721417,365122512,0.0,0.0
3,2004-08-24 00:00:00-04:00,2.767188,2.776143,2.576390,2.608728,304946748,0.0,0.0
4,2004-08-25 00:00:00-04:00,2.610968,2.686591,2.584102,2.636839,183772044,0.0,0.0


In [10]:
#import os
os.makedirs("data/raw", exist_ok=True)

Now we want to save these DataFrame into a data file, so no need to download it again!

In [11]:
apple_data.to_csv("data/raw/AAPL_stock.csv", index=False)
mic_data.to_csv("data/raw/MSFT_stock.csv", index=False)
google_data.to_csv("data/raw/GOOGL_stock.csv", index=False)

In the next step, we are going to use requests + Beautifulsoup + soup libraries and try to 
download the webpage which contain the revenue data.

In [12]:

a_url = "https://www.macrotrends.net/stocks/charts/AAPL/apple/revenue"
m_url = "https://www.macrotrends.net/stocks/charts/MSFT/microsoft/revenue"
g_url = "https://www.macrotrends.net/stocks/charts/GOOGL/alphabet/revenue"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/114.0.0.0 Safari/537.36"
    )
}

apple_html = requests.get(a_url, headers=headers).text
mic_html = requests.get(m_url, headers=headers).text
google_html = requests.get(g_url, headers=headers).text

app_obj = BeautifulSoup(apple_html, "html.parser")
mic_obj = BeautifulSoup(mic_html,"html.parser")
goo_obj = BeautifulSoup(google_html, "html.parser")


Now that we have our "soup objects," we need to find and fetch tables from it.

In [14]:
app_tables = app_obj.find_all("table", class_="historical_data_table")
print(len(app_tables))

mic_tables = mic_obj.find_all("table", class_="historical_data_table")
print(len(mic_tables))

goo_tables = goo_obj.find_all("table", class_="historical_data_table")
print(len(goo_tables))

4
4
4


4, that means the webpage contains four different HTML tables, and I want to identify which one contains the Quarterly Revenue data you want.

In [15]:
for i, table in enumerate(app_tables):
    print(f"\nTable {i}")
    print(table.get_text()[:300])  # show just the first 300 characters



Table 0



Apple Annual Revenue(Millions of US $)




2024
$391,035


2023
$383,285


2022
$394,328


2021
$365,817


2020
$274,515


2019
$260,174


2018
$265,595


2017
$229,234


2016
$215,639


2015
$233,715


2014
$182,795


2013
$170,910


2012
$156,508


2011
$108,249


2010
$65,225


2009
$42,905




Table 1



Apple Quarterly Revenue(Millions of US $)




2025-03-31
$95,359


2024-12-31
$124,300


2024-09-30
$94,930


2024-06-30
$85,777


2024-03-31
$90,753


2023-12-31
$119,575


2023-09-30
$89,498


2023-06-30
$81,797


2023-03-31
$94,836


2022-12-31
$117,154


2022-09-30
$90,146


2022-06-30
$82,95

Table 2



Sector
Industry
Market Cap
Revenue




Computer and Technology
Computer - Microcomputers
$3172.519B
$391.035B



Apple's business primarily runs around its flagship iPhone. However, the Services portfolio that includes cloud services, App store, Apple Music, AppleCare, Apple Pay & licensing and o

Table 3



Stock Name
Country
Market Cap
PE Ratio




Dell (DELL)
Uni

Now that I figured it out the information I want is in table 1, lets enter them into a DataFram